<a href="https://colab.research.google.com/github/groovallstar/test2/blob/master/Porto_Pilsa_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 경진대회: https://www.kaggle.com/c/porto-seguro-safe-driver-prediction
- 원본 커널: https://www.kaggle.com/aharless/xgboost-cv-lb-284  
- 원본 저자: https://www.kaggle.com/aharless
- 참고: https://kaggle-kr.tistory.com/32 (이유한님의 캐글 필사 커리큘럼입니다.)  
- 번역: 홍정호(https://blog.naver.com/h0609zxc, KQNG), 번역 오류가 있다면 언제든지 말씀해주세요
- 필사 날짜: 2019-07-12

훌륭한 커널을 만들어주신 Andy Harless, 이런 커널들을 정리해주신 이유한님께 감사드립니다.  

임의로 가독성을 위해 글의 열, 링크 등은 본문과 조금씩 다릅니다!   
번역을 하면서 저도 모르는 부분이 존재하기 때문에 명쾌하지 않은 부분은 찾으면서 보시면 더 좋을것 같습니다.  
모두 화이팅입니다!

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!ls "/gdrive/My Drive/COLAB/PortoSafeDriverPrediction/"
DATA_PATH = "/gdrive/My Drive/COLAB/PortoSafeDriverPrediction/"

'Porto Pilsa 1.ipynb'   sample_submission.csv   train.csv
'Porto Pilsa 2.ipynb'   test.csv


**이 커널은 olivier님의 커널을 기반으로 만들었습니다.**  
olivier님의 커널: https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

In [ ]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

# 나는 스스로 판단할 수 있는 많은 정보를 얻기 위해 EARLY_STOPPING_ROUNDS를 높게 설정합니다.(OPTIMIZE_ROUNDS가 설정된 경우)
# 만약 실제로 조기 종료를 원한다면 EARLY_STOPPING_ROUNDS를 줄여야합니다.

처음에는 MAX_ROUNDS를 상당히 높게 설정하고 OPTIMIZE_ROUNDS를 사용하여 적당한 라운드 수를 찾는것을 추천합니다.  
(제 판단에는,  모든 폴드 사이에서 best_ntree_limit의 최댓값에 가깝게 해야합니다. 모델이 적절히 정규화 되었거나   
verbose=True를 설정하고,  모든 폴드에서 잘 동작하는 라운드 수를 찾기 위한 세부 정보를 본다면 더 높을 수도 있습니다.)  
그 다음 OPTIMIZE_ROUNDS를 중지하고, MAX_ROUND를 적절한 총 라운드 수로 설정하세요.

각 폴드마다 최적의 라운드를 선택하는 "조기 종료" 방식은 검증 데이터에 과적합하는 문제가 존재합니다.  
따라서 테스트 데이터를 예측하기 위한 최적의 모델을 생성하지 못할 수 있고,   
다른 모델과 스태킹/앙상블을 위해 검증 데이터를 생성하는 경우 앙상블에 너무 많은 weight를 갖게 할 것입니다.  
또 다른 문제점은(XGBoost의 기본 값은) 가장 좋은 라운드보다 조기 종료가 발생한 라운드(개선되지 않고 정체되있는 라운드)를 사용하는 것입니다.    
정체되는 구간이 충분히 길다는 가정이 있다면 이것은 과적합 문제를 해결해줍니다. 그러나 지금까지는 도움이 되지 않았습니다.  
(모든 폴드에 대해 일정한 라운드로 진행한것보다 폴드 당 20라운드 조기 종료를 한 것이 검증 점수가 안좋았다.  그래서 조기 종료는 실제로는 underfit 처럼 보였다.)


In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## gini 계산
## 참고한 링크: https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [ ]:
# 함수는 olivier 커널에서 가져왔습니다.
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))
  
def target_encode(trn_series=None,  
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing은 Daniele Micci-Barreca의 논문에서와 같이 계산됩니다.
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : pd.Series 형태의 학습할 범주형 피처
    tst_series : pd.Series 형태의 테스트할 범주형 피처
    target : pd.Series 형태의 타겟 데이터
    min_samples_leaf (int) : 범주의 평균을 고려할 최소 샘플
    smoothing (int) : 범주 평균과 이전의 균형을 맞추기 위한 스무딩 효과 
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
# 데이터 읽기
train_df = pd.read_csv(DATA_PATH + "train.csv", na_values="-1")
test_df = pd.read_csv(DATA_PATH + "test.csv", na_values="-1")

In [ ]:
# olivier 커널 참고
train_features = [
  "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
# 조합 추가
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [ ]:
# 데이터 처리
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

In [ ]:
y_valid_pred = 0*y
y_test_pred = 0

In [ ]:
# 폴드 설정
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)

In [ ]:
## 분류기 설정
model = XGBClassifier(    
                        n_estimators=MAX_ROUNDS,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=LEARNING_RATE, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3,
                     )

In [ ]:
# CV 실행
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # 폴드에 대한 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)
    
    # 데이터 인코드
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # 폴드에 대한 모델 실행
    if OPTIMIZE_ROUNDS:
        eval_set=[(X_valid,y_valid)]
        fit_model = model.fit( X_train, y_train, 
                               eval_set=eval_set,
                               eval_metric=gini_xgb,
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                               verbose=False
                             )
        print( "  Best N trees = ", model.best_ntree_limit )
        print( "  Best gini = ", model.best_score )
    else:
        fit_model = model.fit( X_train, y_train )
        
    # 폴드에 대한 검증 예측 생성
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    
    # 테스트 세트의 예측을 누적
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K  # 테스트 세트 예측의 평균

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
  Gini =  0.2856959531750338

Fold  1
  Gini =  0.2825270426290394

Fold  2
  Gini =  0.2744124272744569

Fold  3
  Gini =  0.29925913576337726

Fold  4
  Gini =  0.28468083823013424

Gini for full training set:


0.28509183378958614

In [ ]:
# 스태킹 / 앙상블을위한 검증 예측 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv(DATA_PATH + 'xgb_valid.csv', float_format='%.6f', index=False)

In [ ]:
## 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv(DATA_PATH + 'xgb_submit.csv', float_format='%.6f', index=False)